<a href="https://colab.research.google.com/github/ArturArtikov/Atomic_Hack_2.0/blob/main/%D0%AD%D0%BA%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%86%D0%B8%D1%8F%20%D0%B8%20%D0%BE%D1%87%D0%B8%D1%81%D1%82%D0%BA%D0%B0%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0%20%D0%B8%D0%B7%20PDF%20%D1%84%D0%B0%D0%B9%D0%BB%D0%BE%D0%B2.%20%D0%90%D1%82%D0%BE%D0%BC%D0%B8%D0%BA%20%D0%A5%D0%B0%D0%BA%202.0.%20%D0%9A%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4%D0%B0%20Netrunners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install pdfminer
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 28.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.1 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140083 sha256=020524bdfea3fff23cb0a3cc1292e7cece15ee746ab91e1b7aa304f96a6e259c
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.1 MB/s eta 0:00:00


## Экстракция текста из PDF-файлов

In [5]:
from pdfminer.high_level import extract_text
import pandas as pd
import os
import string
import re

Ниже можно увидеть парсер, основанный на применении библиотеки PDFMiner. С ее помощью текст каждого документа делится на куски по 256 слов, что позволяет добавлять его дозирванно в данные, что также помогает более качественно обучать модель

In [6]:
PATH = '/content/drive/MyDrive/AtomicHack/Для Хакатона'
files = os.listdir(PATH)

for name in files:
  text = extract_text(PATH+'/'+name)

  list_of_word = list()
  list_of_text = list()

  for word in text.split():
    if len(list_of_word) < 256:
      list_of_word.append(word)
    else:
      list_of_text.append(list_of_word)
      list_of_word = list()
  list_of_text.append(list_of_word)

  paragraphs = {'Текст файла': list_of_text}
  data = pd.DataFrame(paragraphs)
  data.to_csv(name.replace('.pdf', '.csv'), index=False)

## Предобработка текста

Код ниже получает на вход файлы в формате .csv, после чего проводит их обработку от ненужных символов, а также дополнительно очищает текст от символов, оставшихся после экстракции текста, и не влияющих на него.

In [8]:
folder_path = '/content/drive/MyDrive/AtomicHack/Файлы данных в CSV'

files = os.listdir(folder_path)

def custom_lower(text):
    words = text.split()
    result_words = []

    for word in words:
        if word.isupper():
            result_words.append(word)
        else:
            result_words.append(word.lower())

    return ' '.join(result_words)

def process_text(text):
    text = custom_lower(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[“”«»()\[\]]', '', text)
    text = text.strip('[“”«»()\[\]]')
    return text

for file_name in files:
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)

        data = pd.read_csv(file_path, header=None)

        processed_data = data.applymap(process_text)
        processed_data.to_csv(f'processed_{file_name}', index=False)

        print(f"Предпроцессинг текста для файла {file_name} завершен. Обработанные данные сохранены в:", processed_file_path_name)

print("Предпроцессинг текста для всех файлов завершен.")

Предпроцессинг текста для файла D-1C1-1.05.01 Планирование работы ТС.csv завершен. Обработанные данные сохранены в: /content/drive/My Drive/Processed 2/processed_D-1C1-1.05.01 Планирование работы ТС.csv
Предпроцессинг текста для файла D-1C1-1.18.03 Реализация продукции товаров работ услуг и прочая реализация_v26.csv завершен. Обработанные данные сохранены в: /content/drive/My Drive/Processed 2/processed_D-1C1-1.05.01 Планирование работы ТС.csv
Предпроцессинг текста для файла Инструкция D-1C1-1.01.01 Универсальное согласование документов _v6.csv завершен. Обработанные данные сохранены в: /content/drive/My Drive/Processed 2/processed_D-1C1-1.05.01 Планирование работы ТС.csv
Предпроцессинг текста для файла Инструкция D-1C1-1.01.07 Формирование заявки на платеж.csv завершен. Обработанные данные сохранены в: /content/drive/My Drive/Processed 2/processed_D-1C1-1.05.01 Планирование работы ТС.csv
Предпроцессинг текста для файла Инструкция D-1C1-1.01.09 Формирование платежных поручений (НФ) (5